In [3]:
import kagglehub
import os
import shutil

target_folder = "/home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data"

os.makedirs(target_folder, exist_ok=True) 

print("Đang tải dataset vào cache của kagglehub...")
try:
    cache_path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")
    print(f"Dataset đã được tải về cache tại: {cache_path}")

    print(f"Đang di chuyển tệp từ cache vào thư mục '{target_folder}'...")

    for item_name in os.listdir(cache_path):
        source_item = os.path.join(cache_path, item_name)
        print(f"Soucre item: {source_item}")
        destination_item = os.path.join(target_folder, item_name)

        if os.path.exists(destination_item):
            if os.path.isdir(destination_item):
                shutil.rmtree(destination_item)
            else:
                os.remove(destination_item)

        shutil.move(source_item, destination_item)


    print(f"Hoàn tất! Dữ liệu đã được chuyển đến: {os.path.abspath(target_folder)}")
    print(f"Các tệp trong thư mục '{target_folder}':")
    for item in os.listdir(target_folder):
        print(f"- {item}")

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")
    print("Vui lòng kiểm tra kết nối mạng và xác thực Kaggle của bạn.")
    print("Hãy chắc chắn bạn đã đặt file kaggle.json hoặc dùng kagglehub.login().")

Đang tải dataset vào cache của kagglehub...
Dataset đã được tải về cache tại: /home/tiamo/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2
Đang di chuyển tệp từ cache vào thư mục '/home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data'...
Hoàn tất! Dữ liệu đã được chuyển đến: /home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data
Các tệp trong thư mục '/home/tiamo/Documents/code/NLP/Sentiment-Analysis-Project/data':
- twitter_validation.csv
- twitter_training.csv.zip
- twitter_training.csv


In [12]:
import os
import pandas as pd
path = os.path.join(os.getcwd(),'data/twitter_training.csv')
header = ['label','text']
df = pd.read_csv(path)
df = df.iloc[:,2:4]
df.columns = header
df

,label,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [19]:
import kagglehub
import nltk
import re
import os
import pandas as pd

class TwitterDataset:
    def __init__(self,file_path):
        self.file_path = file_path
        self.df = None
        
    def download_data(self):
        data = pd.read_csv(os.path.join(os.getcwd(),self.file_path))
        header = ['label','text']
        self.df = pd.DataFrame(data.iloc[:,2:4])
        self.df.columns = header
        
    def prepocess_data(self):
        pass
    
twitter = TwitterDataset('data/twitter_training.csv')
twitter.download_data()
twitter.df['label'].value_counts()

label
Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: count, dtype: int64